# Azure OpenAI - Reproducible Output
Jupyter notebook to demo the use of the "**seed**" parameter with openai Python package v1.x

### Pre-requisites:
- You should use Chat Completion API against GPT models on **v1106** or above;
- You should set **seed** to <u>the same integer value</u>;
- You also need to ensure that other parameters like **messages**, **temperature**, etc. <u>stay the same</u>.

In [1]:
"""
Define Azure OpenAI endpoint parameters
    - Set AOAI API Version to env variable OPENAI_API_VERSION
    - Set AOAI API Endpoint to env variable OPENAI_API_BASE
    - Set AOAI API Key to env variable OPENAI_API_KEY
    - Set AOAI Deployment Name to env variable OPENAI_API_DEPLOY
"""

# Importing required packages
from openai import AzureOpenAI
import os

In [2]:
# Initiating Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("OPENAI_API_BASE"),
    api_key = os.getenv("OPENAI_API_KEY"),
    api_version = os.getenv("OPENAI_API_VERSION")
)

AOAI_Deployment = os.getenv("OPENAI_API_DEPLOY")

In [3]:
# Creating subset of prompts
prompts = [
    "Create a story about red panda.",
    "Create a story about red panda."
]

In [4]:
# Defining helper function to generate completions
def generate_completions(prompts, seed = None):
    completions = []

    for prompt in prompts:
        completion = client.chat.completions.create(
            model = AOAI_Deployment, # model = "Azure OpenAI deployment name".
            temperature = 0.1,
            seed = seed,
            messages = [
                {"role": "system", "content": "You always produce 3-sentence answers."},
                {"role": "user", "content": prompt}
            ]        
        )
        fingerprint = completion.system_fingerprint
        content = completion.choices[0].message.content
        completions.append([fingerprint, content])
    
    return completions

In [5]:
# Defining helper function to analyse differences between completions
def analyse_completions(differenciator, completions):
    diff = differenciator.compare(completions[0][1].split(), completions[1][1].split())
    delta = list(delta[2:] for delta in diff if delta.startswith(("+", "-")))

    if delta == []:
        print ("No differences found between completions.")
    else:
        print (f"Found these differences between completions: {delta}.")

### Scenario 1: Testing without seed

In [6]:
# Generating subset of Azure OpenAI completions
completions1 = generate_completions(prompts)

# Printing completions
print("--------------------")
for completion in completions1:
    print(completion[1])
    print("--------------------")

--------------------
In the lush forests of the Himalayas, a curious red panda named Pabu spent his days frolicking among the trees. One day, Pabu stumbled upon a hidden grove filled with the sweetest bamboo he'd ever tasted, but it was guarded by a mischievous monkey. With a clever trick and a daring leap, Pabu outwitted the monkey, securing the grove for his family and becoming the hero of his clan.
--------------------
In the lush forests of the Himalayas, a curious red panda named Pabu spent his days frolicking among the trees. One day, Pabu stumbled upon a hidden grove filled with the sweetest bamboo he had ever tasted, which he decided to keep as his secret snack spot. Little did he know, his delightful discovery would soon attract other forest creatures, leading to unexpected friendships and adventures.
--------------------


### Scenario 2: Testing with seed

In [7]:
# Generating subset of Azure OpenAI completions
completions2 = generate_completions(prompts, seed = 42)

# Printing completions
print("--------------------")
for completion in completions2:
    print(completion[1])
    print("--------------------")

--------------------
In the lush forests of the Himalayas, a curious red panda named Pabu spent his days frolicking among the trees. One day, Pabu stumbled upon a hidden grove filled with the sweetest bamboo he'd ever tasted, but it was guarded by a mischievous monkey. With a clever trick and a daring leap, Pabu outwitted the monkey, securing the grove for his family and becoming the hero of his clan.
--------------------
In the lush forests of the Himalayas, a curious red panda named Pabu spent his days frolicking among the trees. One day, Pabu stumbled upon a hidden grove filled with the sweetest bamboo he'd ever tasted, but it was guarded by a mischievous monkey. With a clever trick and a daring leap, Pabu outwitted the monkey, securing the grove for his family and becoming the hero of his clan.
--------------------


### Checking if output is reproducible

In [8]:
# Initiating differenciator
import difflib as dl

differenciator = dl.Differ()

In [9]:
# Analysis of completions of Scenario 1 (without seed)
analyse_completions(differenciator, completions1)

Found these differences between completions: ["he'd", 'he', 'had', 'but', 'it', 'which', 'he', 'decided', 'to', 'keep', 'was', 'as', 'guarded', 'by', 'a', 'mischievous', 'monkey.', 'With', 'a', 'clever', 'trick', 'his', 'secret', 'snack', 'spot.', 'Little', 'did', 'he', 'know,', 'his', 'delightful', 'discovery', 'would', 'soon', 'attract', 'other', 'forest', 'creatures,', 'leading', 'to', 'unexpected', 'friendships', 'adventures.', 'a', 'daring', 'leap,', 'Pabu', 'outwitted', 'the', 'monkey,', 'securing', 'the', 'grove', 'for', 'his', 'family', 'and', 'becoming', 'the', 'hero', 'of', 'his', 'clan.'].


In [10]:
# Analysis of completions of Scenario 2 (with seed)
analyse_completions(differenciator, completions2)

No differences found between completions.
